In [12]:
import pandas as pd
from dbfread import DBF
path = 'D:\\IPMIProject\\master test\\20231223.dbf' # 文件目录
table = DBF(path, encoding='GBK')
df = pd.DataFrame(iter(table))
print(df)

              BMH ZZLLM   ZZLLMC ZZLLKDDM  ZZLLKDMC ZZLLKCBH ZZLLZWH WGYM  \
0       611689959   199  管理类综合能力     6116      西北大学       99       1  204   
1       510296397   101   思想政治理论     6152    西安翻译学院       11      18  204   
2       610397518   101   思想政治理论     6103    西安石油大学       62      26  201   
3       610797013   101   思想政治理论     6107    西安邮电大学       24       6  201   
4       614698996   101   思想政治理论     6146      西京学院       74      28  204   
...           ...   ...      ...      ...       ...      ...     ...  ...   
172883  653398263   101   思想政治理论     6156    咸阳师范学院      105      11  201   
172884  653399631   101   思想政治理论     6149      安康学院      059      20  201   
172885  653399955   101   思想政治理论     6144    宝鸡文理学院      J31      11  204   
172886  653499532   101   思想政治理论     6141    陕西理工大学      144      28  201   
172887  653499807   101   思想政治理论     6134  西安建筑科技大学      302      09  201   

        WGYMC WGYKDDM  ... YWK2KCBH YWK2ZWH BKDWDM        BKDWMC  BKZYDM  \

In [13]:
# 数据预处理尝试

import numpy as np
import pandas as pd

allowed_values1 = ['101','109','199']
allowed_values2 = ['201','202','203','204']
allowed_values3 = ['301','302','303','306','307','311','312','313','314','315','333','396','397','398']
allowed_values4 = ['408','414','415','497','498']
df.loc[~df['ZZLLM'].isin(allowed_values1), 'ZZLLM'] = '000'
df.loc[~df['WGYM'].isin(allowed_values2), 'WGYM'] = '000'
df.loc[~df['YWK1M'].isin(allowed_values3), 'YWK1M'] = '000'
df.loc[~df['YWK2M'].isin(allowed_values4), 'YWK2M'] = '000'
df.columns = df.columns.map(str.lower)
arr = df
# print(arr.head(10))

# ["bmh","zzllm","wgym","ywk1m","ywk2m","bkdwdm"]
# 取出dataframe的几列
df1 = arr[["bmh","zzllm","wgym","ywk1m","ywk2m","bkdwdm"]]

# 使用其中的四个属性进行分组
df2 = df1.groupby(["zzllm","wgym","ywk1m","ywk2m"]).count()
df2.to_excel("classfier.xlsx")


# sub_DF=[arr[arr['wgymc'].isin([i])] for i in np.unique(arr['wgymc'])]
# print(df2.head())
# groupby后的数据要先转为dataframe，再进行索引重建，才可以保存为xlsx
# class_arr = pd.DataFrame(df1.groupby(["zzllmc","wgymc","ywk1mc","ywk2mc"]))
# class_arr.reset_index(inplace=True)
# class_arr.to_excel("res.xlsx")

In [14]:
# 按照外国语考点代码拆分数据

import numpy as np
import pandas as pd

# arr = pd.read_excel("tmp.xlsx")
list1 = arr['wgykddm'].unique()
groups = arr.groupby("wgykddm")
for i in arr['wgykddm'].unique():
    groups.get_group(i).to_excel(str(i)+".xlsx")
# print(groups['wgykddm'])
    # 检查数量是否相等
# print(arr.shape)#(16383,32)
# sum = 0;
# for i in arr['wgykddm'].unique():
#     df1 = pd.read_excel(str(i)+".xlsx")
#     sum += df1.shape[0]

# print(sum)#16383

In [15]:
# 统计每个考点教室所能容纳的人数
groups1 = pd.read_excel("考场容量表.xls").groupby("zzllkddm")
print(groups1.apply(lambda x:x))
print(groups1.get_group(6103)["renshu"].tolist())

WARNING *** OLE2 inconsistency: SSCS size is 0 but SSAT size is non-zero
*** No CODEPAGE record, no encoding_override: will use 'iso-8859-1'
      zzllkddm zzllkcbh  renshu
0         6103        1      30
1         6103       10      30
2         6103      100      30
3         6103      101      30
4         6103      102      30
...        ...      ...     ...
5420      6160      103      30
5421      6160      104      30
5422      6160      105      30
5423      6160      106      30
5424      6160      107      16

[5425 rows x 3 columns]
[30, 30, 30, 30, 30, 30, 30, 30, 30, 30, 30, 30, 30, 30, 30, 30, 30, 30, 30, 30, 30, 30, 30, 30, 30, 30, 30, 30, 30, 30, 30, 30, 30, 30, 30, 30, 30, 30, 30, 30, 30, 30, 30, 30, 30, 30, 30, 30, 30, 30, 30, 30, 30, 29, 30, 30, 30, 31, 30, 30, 30, 30, 30, 30, 30, 30, 30, 30, 30, 30, 30, 30, 30, 30, 30, 30, 30, 30, 30, 30, 30, 30, 30, 30, 30, 30, 30, 30, 30, 30, 30, 30, 30, 30, 30, 30, 30, 30, 30, 30, 30, 30, 30, 30, 30, 30, 30, 30, 30, 30, 30, 26, 3

In [16]:
# 统计每个考点有多少专业，以及每个专业有多少人，参考属性有["zzllmc","wgymc","ywk1mc","ywk2mc","bkdwdm"]

for i in arr['wgykddm'].unique():
    df1 = pd.read_excel(str(i)+".xlsx")

    # 使用其中的四个属性进行分组,"bkdwdm"
    df2 = df1.groupby(["zzllm","wgym","ywk1m","ywk2m"]).count()
    df2.to_excel("classfier"+str(i)+".xlsx")

# 结果里面bmh表示的是每个专业的人数

In [21]:
# 数据预处理、提取出每个考点，每个专业的人数
sums = 0
print(list1)
for i in list1:
    # .fillna(method="ffill")向下填充
    res = pd.read_excel("classfier"+str(i)+".xlsx").fillna(method="ffill")
    # dataframe按各专业人数一列排序，inplace原地排序，ascending升序
    # res.sort_values(by="bmh",inplace=True,ascending=False)
    # 取出各专业人数一列转为array
    # 若存在控制需要填充
    # res = res.fillna(res['bmh'].mean())
    StuNum = res[['zzllmc','wgymc','ywk1mc','ywk2mc','bmh']]
    np.savetxt("classfier"+str(i)+".txt",StuNum,fmt="%s,%s,%s,%s,%s",delimiter=" ")
    sums += res['bmh'].sum()

print(sums)

# classfierX.excel中的前四列可以映射到对应的专业

['6116' '6152' '6103' '6107' '6146' '6144' '6143' '6139' '6117' '6153'
 '6118' '6160' '6141' '6114' '6113' '6124' '6134' '6105' '6136' '6149'
 '6159' '6157' '6151' '6115' '6112' '6111' '6156' '6133' '6148' '6140'
 '6125' '6154' '6147' '6150' '6145' '6127' '6138' '6128' '6122' '6129'
 '6130' '6131' '6142' '6158']
172888


In [18]:
def Sum_matrix(matrix):
    sum=0
    for i in range(len(matrix)):
        for j in range(len(matrix[i])):
            sum+=matrix[i][j]
    return sum

In [33]:
import pandas as pd
pd.options.display.max_rows = None
# 分组算法
# pd.set_option('display.max_rows',200)
# StuNum = np.loadtxt("classfier6127.txt")
aa=pd.read_excel("classfier6105.xlsx").iloc[:,:5]
aa.columns=['s1','s2','s3','s4','num']
print(aa)
# 对txt文件的数据排序，方便后面索引对应
print("各专业人数")
print(aa.sort_values(by='num',axis=0).reset_index(drop=True))
StuNum = pd.read_excel("classfier6105.xlsx",header=None).iloc[:,4].tolist()[1:]
print(StuNum)

# 设置参数：教室最多可容纳人数、考场允许最大专业数量
max_student=groups1.get_group(6105)["renshu"].tolist()
print("考场容量：",max_student)
print("考场数量：",len(max_student))
max_subject=5
# 存放分组结果
groups = []

# 这里还没有考虑取余的情况
for i in range(len(StuNum)):
    StuNum[i] = StuNum[i] % 30

StuNum.sort()
# print(len(StuNum))
start = 0
end = len(StuNum)-1
classroom = 0
while(start<end and classroom<len(max_student)):
    class_subject = []#用来存储人数对应的索引，输出的就是这个，和上面txt文件的输出对应
    class_sum = StuNum[end]#每次循环初始，让sum等于尾指针指向的数
    subject = 1
    class_subject.append(end)
    # 人数不超过教室容量、专业不超过教室容量、专业还未分完
    while(class_sum<max_student[classroom] and subject<max_subject and start<end):
        class_sum = class_sum + StuNum[start]
        class_subject.append(start)
        start = start+1
        subject = subject+1
    end = end-1
    classroom+=1
    groups.append(class_subject)

print(groups)
print(len(groups))
# 下面的数字是不同袋子的编号

       s1     s2     s3   s4   num
0   101.0    0.0    0.0    0    67
1     NaN    NaN  301.0    0     1
2     NaN  201.0    0.0    0   747
3     NaN    NaN  301.0    0   835
4     NaN    NaN    NaN  408    37
5     NaN    NaN  302.0    0   153
6     NaN    NaN    NaN  408     5
7     NaN    NaN  303.0    0   169
8     NaN    NaN  306.0    0    30
9     NaN    NaN  307.0    0    16
10    NaN    NaN  311.0    0    13
11    NaN    NaN  312.0    0    24
12    NaN    NaN  313.0    0    13
13    NaN    NaN  314.0    0     1
14    NaN    NaN  315.0    0     1
15    NaN    NaN    NaN  414     5
16    NaN    NaN    NaN  415     3
17    NaN    NaN  396.0    0     9
18    NaN    NaN  397.0  497     9
19    NaN    NaN  398.0  498    60
20    NaN  202.0    0.0    0     1
21    NaN    NaN  302.0    0     1
22    NaN  203.0    0.0    0     6
23    NaN    NaN  301.0    0     1
24    NaN    NaN  302.0    0     1
25    NaN    NaN  312.0    0     1
26    NaN    NaN  313.0    0     1
27    NaN  204.0    

In [15]:
# 分组算法
import os
list1 = []
names = os.listdir("./")
for name in names:
    if name.endswith('.txt'):
        list1.append(name)
print(list1)

summ=0
for j in list1:
    # 读取文件并定位到人数一列
    if j=='classfier6127.txt':
        print(j)
        # header=None表示没有列标签
    StuNum = pd.read_csv(j,encoding='gb2312',header=None).iloc[:,0]
    # print(StuNum)

    # 设置参数：教室最多可容纳人数、考场允许最大专业数量
    max_student=30
    max_subject=5
    groups = []
    dic = {}
    # 建立原始值 和 余数的映射
    for i in range(len(StuNum)):
        dic[StuNum[i]] = StuNum[i] % 30

    # 保存字典映射关系
    StuNum = []
    with open('params.txt', 'w') as f:
        for key, value in dic.items():
            f.write(str(key))
            f.write(': ')
            f.write(str(value))
            f.write('\n')
            StuNum.append(value)


    # 对余数进行排序
    # dic = sorted(dic.items(),key=lambda x:x[1],reverse=False)
    StuNum.sort()
    # print(len(StuNum))
    start = 0
    end = len(StuNum)-1
    while(start<end):
        class_subject = []
        class_sum = StuNum[end]
        subject = 1
        class_subject.append(end)
        # ===================================================文件6127在这里出了问题
        while(class_sum<max_student and subject<max_subject):
            class_sum = class_sum + StuNum[start]
            class_subject.append(start)
            start = start+1
            subject = subject+1
        end = end-1
        groups.append(class_subject)

    print(groups)
    print(j)
    summ=summ+Sum_matrix(groups)
    print("\n")
print(summ)
    # 下面的数字是不同袋子的编号

['classfier6103.txt', 'classfier6105.txt', 'classfier6107.txt', 'classfier6111.txt', 'classfier6112.txt', 'classfier6113.txt', 'classfier6114.txt', 'classfier6115.txt', 'classfier6116.txt', 'classfier6117.txt', 'classfier6118.txt', 'classfier6122.txt', 'classfier6124.txt', 'classfier6125.txt', 'classfier6127.txt', 'classfier6128.txt', 'classfier6129.txt', 'classfier6130.txt', 'classfier6131.txt', 'classfier6133.txt', 'classfier6134.txt', 'classfier6136.txt', 'classfier6138.txt', 'classfier6139.txt', 'classfier6140.txt', 'classfier6141.txt', 'classfier6142.txt', 'classfier6143.txt', 'classfier6144.txt', 'classfier6145.txt', 'classfier6146.txt', 'classfier6147.txt', 'classfier6148.txt', 'classfier6149.txt', 'classfier6150.txt', 'classfier6151.txt', 'classfier6152.txt', 'classfier6153.txt', 'classfier6154.txt', 'classfier6156.txt', 'classfier6157.txt', 'classfier6158.txt', 'classfier6159.txt', 'classfier6160.txt', 'params.txt']


ParserError: Error tokenizing data. C error: Expected 6 fields in line 804, saw 8
